In [38]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import statsmodels.api as sm

# Set plotting style
plt.style.use("default")
params = {
    "axes.labelsize": 12, "font.size": 12, "legend.fontsize": 12,
    "xtick.labelsize": 12, "ytick.labelsize": 12, "text.usetex": False,
    "font.family": "sans-serif", "axes.spines.top": False, "axes.spines.right": False,
    "grid.color": "grey", "axes.grid": True, "grid.alpha": 0.5, "grid.linestyle": ":",
}
plt.rcParams.update(params)

# --------------------------
# User Settings and Parameters
# --------------------------
market_index = 'SPY'   # For beta calculation
start_date = "2006-01-01"
end_date   = "2023-12-31"

# List of factors to include in the regression.
selected_factors = ['Mkt-RF', 'SMB', 'HML', 'Beta', 'Volatility', 'Momentum', 'Reversal', 'Value', 'Growth']

# --------------------------
# Tickers
# --------------------------
tickers = ['CASY', 'NVDA', 'META', 'TSLA']

# --------------------------
# Download Price Data for S&P 500 and the Market Index
# --------------------------
prices = yf.download(tickers, start=start_date, end=end_date)["Close"]
market_data = yf.download(market_index, start=start_date, end=end_date)

# --------------------------
# Resample to Monthly End and Compute Returns
# --------------------------
# Use month-end frequency ("ME") and forward-fill missing values
stock_monthly = prices.resample("ME").ffill()
market_monthly = market_data["Close"].resample("ME").ffill()

# Convert indices to PeriodIndex (monthly) for consistency with factor data
stock_monthly.index = stock_monthly.index.to_period("M")
market_monthly.index = market_monthly.index.to_period("M")

# Compute monthly returns in percentage terms
stock_returns = stock_monthly.pct_change() * 100
market_returns = market_monthly.pct_change() * 100
market_returns = pd.DataFrame(market_returns)
market_returns.rename(columns={"Close": "Price"}, inplace=True)
# Rename market return column for clarity
market_returns.rename(columns={market_returns.columns[0]: "SPY_Return"}, inplace=True)

# --------------------------
# Load Fama-French Factor Data
# --------------------------
ff_factors = pd.read_csv("F-F_Research_Data_Factors-monthly.CSV", index_col=0)
ff_factors.index.name = "Date"
ff_factors.index = pd.to_datetime(ff_factors.index, format="%Y%m")
ff_factors.index = ff_factors.index.to_period("M")
# Filter to dates that overlap with our stock returns
ff_factors = ff_factors[ff_factors.index.isin(stock_returns.index)].copy()

# --------------------------
# Prepare to Loop Over Each Ticker
# --------------------------
# Dictionary to store regression models for each ticker
regression_results = {}

# Define a helper function for momentum calculation:
def cumulative_return(x):
    # Convert percentage returns to decimals, compute cumulative product, then back to percentage
    return (np.prod(1 + x/100) - 1) * 100

# --------------------------
# Loop Over S&P 500 Tickers and Run Factor Analysis
# --------------------------
for ticker in tickers:
    print(f"Processing {ticker}...")
    
    # Create a DataFrame for the ticker's monthly returns
    df = pd.DataFrame()
    df["Return"] = stock_returns[ticker]
    
    # Merge with market returns (SPY) using the PeriodIndex
    df = df.merge(market_returns[["SPY_Return"]], left_index=True, right_index=True, how="inner")
    # Merge with Fama-French factors
    df = df.merge(ff_factors, left_index=True, right_index=True, how="inner")
    
    # Calculate Excess Return: stock return minus risk-free rate (RF from Fama-French data)
    df["Excess_Return"] = df["Return"] - df["RF"]
    
    # Compute Additional Factors:
    # Beta: 36-month rolling beta using SPY returns
    df["Beta"] = df["Return"].rolling(window=36).cov(df["SPY_Return"]) / df["SPY_Return"].rolling(window=36).var()
    
    # Volatility: 36-month rolling standard deviation (monthly terms)
    df["Volatility"] = df["Return"].rolling(window=36).std()
    
    # Momentum: cumulative return from month t-12 to t-2 (skip most recent month)
    # We shift by 1 to skip the most recent month, then use a rolling window of 11 months.
    df["Momentum"] = df["Return"].shift(1).rolling(window=11).apply(cumulative_return, raw=True)
    
    # Reversal: previous month's return
    df["Reversal"] = df["Return"].shift(1)
    
    # Value and Growth: fetch current fundamentals from yfinance and use them as constant factors.
    try:
        ticker_obj = yf.Ticker(ticker)
        info = ticker_obj.info
        value_factor = info.get("priceToBook", np.nan)
        growth_factor = info.get("earningsQuarterlyGrowth", np.nan)
    except Exception as e:
        print(f"Error fetching fundamentals for {ticker}: {e}")
        value_factor, growth_factor = np.nan, np.nan
        
    df["Value"] = value_factor
    df["Growth"] = growth_factor
    
    # Drop rows with any missing values (due to rolling calculations or fundamental data)
    df.dropna(inplace=True)
    
    if df.empty:
        print(f"Insufficient data for {ticker}; skipping regression.")
        continue
    
    # --------------------------
    # Regression: Prepare Dependent and Independent Variables
    # --------------------------
    y_var = df["Excess_Return"]
    # Use only the factors that are available in our DataFrame and that the user selected
    available_factors = df.columns.intersection(selected_factors)
    X = df[available_factors]
    X = sm.add_constant(X)
    
    # Run the OLS regression
    model = sm.OLS(y_var, X).fit()
    regression_results[ticker] = model  # Store the fitted model if needed later
    
    # Print a brief summary for this ticker
    print(model.summary())
    print("\n" + "="*80 + "\n")

# At this point, regression_results contains the model for each ticker that had sufficient data.

[*********************100%***********************]  4 of 4 completed
[*********************100%***********************]  1 of 1 completed


Processing CASY...
                            OLS Regression Results                            
Dep. Variable:          Excess_Return   R-squared:                       0.269
Model:                            OLS   Adj. R-squared:                  0.239
Method:                 Least Squares   F-statistic:                     9.049
Date:                Tue, 11 Mar 2025   Prob (F-statistic):           1.72e-09
Time:                        23:20:39   Log-Likelihood:                -569.18
No. Observations:                 180   AIC:                             1154.
Df Residuals:                     172   BIC:                             1180.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Mkt-RF         0.6509      0.105 